In [0]:
# Databricks notebook source
# MAGIC %md
# MAGIC # 02_stage_model
# MAGIC 
# MAGIC - MLflow에 등록된 Realtime 모델을 Staging으로 이동

# COMMAND ----------
import os


from mlflow.tracking import MlflowClient
import logging

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# ----------------------
# 설정
# ----------------------
REALTIME_MODEL_NAME = os.getenv("REALTIME_MODEL_NAME","team4-realtime-model")
client = MlflowClient()

# COMMAND ----------

# 1️⃣ 최신 모델 버전 가져오기
try:
    latest_versions = client.get_latest_versions(REALTIME_MODEL_NAME, stages=["None"])
    
    if not latest_versions:
        print(f"❌ '{REALTIME_MODEL_NAME}'의 None 단계 모델을 찾을 수 없습니다.")
    else:
        latest_version = latest_versions[0]
        version_number = latest_version.version
        print(f"✅ 최신 모델 버전: {version_number}")
except Exception as e:
    print(f"❌ 모델 버전 확인 실패: {e}")

# COMMAND ----------

# 2️⃣ Staging으로 이동
try:
    client.transition_model_version_stage(
        name=REALTIME_MODEL_NAME,
        version=version_number,
        stage="Staging",
        archive_existing_versions=False
    )
    print(f"✅ 모델 버전 {version_number}을 Staging으로 이동 완료!")
except Exception as e:
    print(f"❌ 모델 스테이지 이동 실패: {e}")
